# Hosting Strands + LangGraph agent with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a LangGraph with Amazon Bedrock model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model)
and for a Strands Agents with an OpenAI model check [here](../03-strands-with-openai-model).

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | Strands + LangGraph                                                                    |
| LLM model           | Anthropic Claude Sonnet 3                                                    |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands + LangGraph and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a LangGraph agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

In [ ]:
#!uv add -r requirements.txt --active

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

In [3]:
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

In [4]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


In [5]:
# Create the graph
graph = build_graph()

In [6]:
def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

In [7]:
import nest_asyncio
nest_asyncio.apply()

#remove_artifact_folder()

user_query = '''
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 
'''

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)



INFO [__main__] ===== Starting workflow =====

INFO [__main__] 
user input: 
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 


INFO [graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!

데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.

handoff_to_planner: I'll need to consult our planning system for this request.


INFO [graph.nodes] ===== Coordinator completed task =====

INFO [graph.nodes] ===== Planner generating full plan =====

INFO [src.utils.strands_sdk_utils] PLANNER - Prompt Cache Enabled


Let's analyze the user's request in Korean and create an appropriate plan.

The user says:
1. "안녕 나는 장동진이라고 해. 만나서 반가워" (Hello, my name is Jang Dongjin. Nice to meet you.)
2. "나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어." (I want to provide data and extract insights from it.)

This is a general request about data analysis and insight extraction. However, the user hasn't provided any specific data yet, so I'll need to create a plan that first asks the user to provide data, and then outlines the steps needed to extract insights from that data.

Let me create a comprehensive plan in Korean since the user communicated in Korean:

1. We need to understand what kind of data the user has and wants to analyze.
2. We need to process and analyze that data.
3. We need to report insights derived from the data analysis.

Since the user hasn't provided the data yet, I'll create a plan that starts by asking for the data and then outlines the general process for data analysis and insight extraction.# Plan

## 


INFO [graph.nodes] ===== Planner completed task =====

INFO [__main__] ===== Workflow completed successfully =====


In [34]:
%%writefile strands_langgraph_bedrock.py

import sys, os
#module_path = "../../../.."
#module_path = "../../../.."
#sys.path.append(os.path.abspath(module_path))

import json
import logging
import argparse
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


# Create the graph
graph = build_graph()

def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

def strands_langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_query = payload.get("prompt")

    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    # Create the input in the format expected by LangGraph
    #response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    #return response["messages"][-1].content
    return result

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_langgraph_bedrock(json.loads(args.payload))
    print(response)



Overwriting strands_langgraph_bedrock.py


#### Invoking local agent

In [35]:
!python strands_langgraph_bedrock.py '{"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."}'


INFO [__main__] ===== Starting workflow =====

INFO [__main__] 
user input: 안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어.

INFO [graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled
안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!

데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.

handoff_to_planner: I'll need to consult our planning system for this request.
INFO [graph.nodes] ===== Coordinator completed task =====

INFO [graph.nodes] ===== Planner generating full plan =====

INFO [src.utils.strands_sdk_utils] PLANNER - Prompt Cache Enabled
Let me analyze the user request and plan the appropriate steps to fulfill it.

The user's message is in Korean, and it translates roughly to: "Hello, my name is Jang Dongjin. Nice to meet you. I want to provide data and extract insights from it."

The user introduces themselves and expresses a desire to provide data for analysis to extract insights. However, the user 

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agent SDK + LangGraph with Amazon Bedrock model
Let's start with our Strands Agent SDK + LangGraph using Amazon Bedrock model. Other examples with different frameworks and models are available in the parent directories

### Dockerfile


In [36]:
%%writefile strands_langgraph_bedrock.py

import sys, os
#module_path = "../../../.."
#module_path = "../../../.."
#sys.path.append(os.path.abspath(module_path))

import json
import logging
import argparse
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


# Create the graph
graph = build_graph()

def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

@app.entrypoint
def strands_langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_query = payload.get("prompt")

    result = run_agent_workflow(
        user_input=user_query,
        debug=False
    )
    
    # Create the input in the format expected by LangGraph
    #response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    #return response["messages"][-1].content
    return result

if __name__ == "__main__":
    app.run()
    
    #parser = argparse.ArgumentParser()
    #parser.add_argument("payload", type=str)
    #args = parser.parse_args()
    #response = strands_langgraph_bedrock(json.loads(args.payload))
    #print(response)



Overwriting strands_langgraph_bedrock.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [37]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="strandlanggraph_bedrock"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials
Role already exists -- deleting and creating it again
policies: {'PolicyNames': ['AgentCorePolicy'], 'IsTruncated': False, 'ResponseMetadata': {'RequestId': '9dcfc1bd-1eed-4280-8b2a-a587fd2bea4c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 08:53:18 GMT', 'x-amzn-requestid': '9dcfc1bd-1eed-4280-8b2a-a587fd2bea4c', 'content-type': 'text/xml', 'content-length': '380'}, 'RetryAttempts': 0}}
deleting agentcore-strandlanggraph_bedrock-role
recreating agentcore-strandlanggraph_bedrock-role
attaching role policy agentcore-strandlanggraph_bedrock-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [15]:
agentcore_runtime.configure?

Signature:
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
Docstring:
Configure Bedrock AgentCore from notebook using an entrypoint file.

Args:
    entrypoint: Path to Python file with optional Bedrock AgentCore name
        (e.g., "handler.py" or "handler.py:bedrock_agentcore")
    execution_role: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
    agent_name: name of the agent
    require

In [43]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/strands_langgraph_bedrock.py, bedrock_agentcore_name=strands_langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/strands_langgraph_bedrock.py, bedrock_agentcore_name=strands_langgraph_bedrock
Configuring BedrockAgentCore agent: strands_langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: strands_langgraph_bedrock
Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile
INFO:bedro

ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-strandlanggraph_bedrock-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [44]:
launch_result = agentcore_runtime.launch()

Launching Bedrock AgentCore agent 'strands_langgraph_bedrock' to cloud
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Launching Bedrock AgentCore agent 'strands_langgraph_bedrock' to cloud


Build: Successfully installed MarkupSafe-3.0.2 PyYAML-6.0.2 SQLAlchemy-2.0.42 annotated-types-0.7.0 anyio-4.10.0 argon2-cffi-25.1.0 argon2-cffi-bindings-25.1.0 arrow-1.3.0 asttokens-3.0.0 async-lru-2.0.5 attrs-25.3.0 aws-requests-auth-0.4.3 awscli-1.42.1 babel-2.17.0 beautifulsoup4-4.13.4 bedrock-agentcore-0.1.1 bedrock-agentcore-starter-toolkit-0.1.3 bleach-6.2.0 boto3-1.40.1 botocore-1.40.1 certifi-2025.8.3 cffi-1.17.1 charset_normalizer-3.4.2 click-8.2.1 colorama-0.4.6 comm-0.2.3 debugpy-1.8.15 decorator-5.2.1 defusedxml-0.7.1 dill-0.4.0 docstring-parser-0.17.0 docutils-0.19 executing-2.2.0 fastjsonschema-2.21.1 fqdn-1.5.1 greenlet-3.2.3 h11-0.16.0 html5lib-1.1 httpcore-1.0.9 httpx-0.28.1 httpx-sse-0.4.1 idna-3.10 importlib-metadata-8.7.0 ipykernel-6.30.0 ipython-9.4.0 ipython-pygments-lexers-1.1.1 ipywidgets-8.1.7 isoduration-20.11.0 jedi-0.19.2 jinja2-3.1.6 jmespath-1.0.1 json5-0.12.0 jsonpatch-1.33 jsonpointer-3.0.0 jsonschema-4.25.0 jsonschema-specifications-2025.4.1 jupyter-1.1

✅ Reusing existing ECR repository: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock


Authenticating with registry...
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Authenticating with registry...
Registry authentication successful
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Registry authentication successful
Tagging image: bedrock_agentcore-strands_langgraph_bedrock:latest -> 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock:latest
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Tagging image: bedrock_agentcore-strands_langgraph_bedrock:latest -> 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock:latest
Pushing image to registry...
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Pushing image to registry...


The push refers to repository [615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock]
9c2e489caf33: Preparing
8069433ef2a8: Preparing
630c4be75fed: Preparing
5a2b9050aa4c: Preparing
6ed6547a37cd: Preparing
2e763e579d90: Preparing
05ed7b2bcaa5: Preparing
d74514903961: Preparing
e26286284f6e: Preparing
dd97e58b4e81: Preparing
e26286284f6e: Waiting
2e763e579d90: Waiting
05ed7b2bcaa5: Waiting
dd97e58b4e81: Waiting
d74514903961: Waiting
6ed6547a37cd: Pushed
8069433ef2a8: Pushed
2e763e579d90: Layer already exists
05ed7b2bcaa5: Layer already exists
d74514903961: Layer already exists
9c2e489caf33: Pushed
e26286284f6e: Layer already exists
dd97e58b4e81: Layer already exists
630c4be75fed: Pushed
5a2b9050aa4c: Pushed


Image pushed successfully
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.container:Image pushed successfully
Image uploaded to ECR: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Image uploaded to ECR: 615299776985.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_langgraph_bedrock
Deploying to Bedrock AgentCore...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Deploying to Bedrock AgentCore...


latest: digest: sha256:255419f1831b0cd4b6a90c6bea927ddde03ca1d1bb5c99e53c66f86405f8dcea size: 2415


✅ Agent created/updated: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_bedrock-cX1ijBFQs8
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:✅ Agent created/updated: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_bedrock-cX1ijBFQs8
Polling for endpoint to be ready...
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Polling for endpoint to be ready...
Agent endpoint: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_bedrock-cX1ijBFQs8/runtime-endpoint/DEFAULT
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Agent endpoint: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_bedrock-cX1ijBFQs8/runtime-endpoint/DEFAULT
Deployed to cloud: arn:aws:bedrock-agentcore:us-west-2:615299776985:runtime/strands_langgraph_bedrock-cX1ijBFQs8
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Deployed to cloud: arn:aws:bedrock-age

In [45]:
import platform

print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: aarch64
Processor: aarch64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1033-aws-aarch64-with-glibc2.35


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [46]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore:Retrieved Bedrock AgentCore status for: strands_langgraph_bedrock


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [56]:
#invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response = agentcore_runtime.invoke({"prompt": "안녕 나는 장동진이라고 해. 만나서 반가워 나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어."})
invoke_response

Invoking BedrockAgentCore agent 'strands_langgraph_bedrock' via cloud endpoint
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.invoke:Invoking BedrockAgentCore agent 'strands_langgraph_bedrock' via cloud endpoint


{'ResponseMetadata': {'RequestId': '6d060398-3b5a-4284-884d-64bafc5c0f67',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 04 Aug 2025 09:15:52 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6d060398-3b5a-4284-884d-64bafc5c0f67',
   'baggage': 'Self=1-68907a35-7cacfb55363afae112c16f0a,session.id=efcc3c0d-7c7e-4d19-b1d1-b575efced24a',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'efcc3c0d-7c7e-4d19-b1d1-b575efced24a',
   'x-amzn-trace-id': 'Root=1-68907a35-67c800110c29688a49e2a037;Self=1-68907a35-7cacfb55363afae112c16f0a'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'efcc3c0d-7c7e-4d19-b1d1-b575efced24a',
 'traceId': 'Root=1-68907a35-67c800110c29688a49e2a037;Self=1-68907a35-7cacfb55363afae112c16f0a',
 'baggage': 'Self=1-68907a35-7cacfb55363afae112c16f0a,session.id=efcc3c0d-7c7e-4d19-b1d1-b575efced24a',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': [b'{"full_plan"

In [ ]:
스트림

https://strandsagents.com/latest/documentation/docs/user-guide/deploy/deploy_to_bedrock_agentcore/

In [ ]:
invoke_response["response"][0]

'b\'{"full_plan": "# Plan\\\\n## thought\\\\n  - \\xec\\x82\\xac\\xec\\x9a\\xa9\\xec\\x9e\\x90\\xeb\\x8a\\x94 \\xed\\x95\\x9c\\xea\\xb5\\xad\\xec\\x96\\xb4\\xeb\\xa1\\x9c \\xeb\\x8c\\x80\\xed\\x99\\x94\\xed\\x95\\x98\\xeb\\xa9\\xb0, \\xeb\\x8d\\xb0\\xec\\x9d\\xb4\\xed\\x84\\xb0\\xeb\\xa5\\xbc \\xec\\xa0\\x9c\\xea\\xb3\\xb5\\xed\\x95\\x98\\xea\\xb3\\xa0 \\xea\\xb7\\xb8\\xeb\\xa1\\x9c\\xeb\\xb6\\x80\\xed\\x84\\xb0 \\xec\\x9d\\xb8\\xec\\x82\\xac\\xec\\x9d\\xb4\\xed\\x8a\\xb8\\xeb\\xa5\\xbc \\xec\\xb6\\x94\\xec\\xb6\\x9c\\xed\\x95\\x98\\xea\\xb3\\xa0 \\xec\\x8b\\xb6\\xeb\\x8b\\xa4\\xea\\xb3\\xa0 \\xec\\x9a\\x94\\xec\\xb2\\xad\\xed\\x96\\x88\\xec\\x8a\\xb5\\xeb\\x8b\\x88\\xeb\\x8b\\xa4.\\\\n  - \\xed\\x95\\x98\\xec\\xa7\\x80\\xeb\\xa7\\x8c \\xec\\x95\\x84\\xec\\xa7\\x81 \\xea\\xb5\\xac\\xec\\xb2\\xb4\\xec\\xa0\\x81\\xec\\x9d\\xb8 \\xeb\\x8d\\xb0\\xec\\x9d\\xb4\\xed\\x84\\xb0\\xeb\\x82\\x98 \\xeb\\xb6\\x84\\xec\\x84\\x9d \\xec\\x9a\\x94\\xec\\xb2\\xad\\xec\\x9d\\xb4 \\xec\\xa0\\x9c\\xea\\xb3

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!